# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [41]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nhl_df.drop([0,9,18,26],0,inplace=True)
    cities.drop([14,15,18,19,20,21,23,24,25,27,28,32,33,38,40,41,42,44,45,46,48,49,50],0,inplace=True)
    cities['NHL']=cities['NHL'].str.replace("\[.*\]","")
    cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                            "KingsDucks": "Kings,Ducks",
                                            "Red Wings": "Red,Wings", 
                                            "Maple Leafs": "Maple,Leafs", 
                                            "Blue Jackets": "Blue,Jackets",
                                            "Golden Knights": "Golden,Knights" })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")

    nhl_df=nhl_df[nhl_df["year"]==2018]
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", ""))
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])

    merge=pd.merge(cities,nhl_df,left_on="NHL",right_on="team")
    merge = merge[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
    merge["win-loss"]=(merge["W"].astype(int)/(merge["W"].astype(int)+merge["L"].astype(int))).astype(float)
    merge["Population (2016 est.)[8]"] = merge["Population (2016 est.)[8]"].astype("float")

    merge.loc[merge["Metropolitan area"] == "New York City", "win-loss"] = 0.51820142
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "win-loss"] = 0.622894635 
    merge = merge.drop_duplicates(subset="Metropolitan area").reset_index()

    
    
    population_by_region = merge["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = merge["win-loss"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [42]:
nhl_correlation()

C:\Users\KIIT\AppData\Local\Temp\ipykernel_544\2369599485.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  nhl_df.drop([0,9,18,26],0,inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_544\2369599485.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cities.drop([14,15,18,19,20,21,23,24,25,27,28,32,33,38,40,41,42,44,45,46,48,49,50],0,inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_544\2369599485.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NHL']=cities['NHL'].str.replace("\[.*\]","")


0.012486163034016697

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [34]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities.drop([16,17,19,20,21,22,23,26,29,30,31,34,35,36,37,39,40,43,44,47,48,49,50],0,inplace=True)
    cities['NBA']=cities['NBA'].str.replace("\[.*\]","")
    cities["NBA"] = cities["NBA"].replace({"KnicksNets": "Knicks,Nets",
                                            "LakersClippers": "Lakers,Clippers",
                                            "Trail Blazers": "Trail,Blazers"})
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
    cities = cities.explode("NBA")

    nba_df=nba_df[nba_df["year"]==2018]
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])

    merge=pd.merge(cities,nba_df,left_on="NBA",right_on="team")
    merge = merge[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L","W/L%"]]
    merge["Population (2016 est.)[8]"] = merge["Population (2016 est.)[8]"].astype("float")
    merge["W/L%"]=merge["W/L%"].astype(float)

    merge.loc[merge["Metropolitan area"] == "New York City", "W/L%"] = 0.3475
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "W/L%"] = 0.4695 
    merge = merge.drop_duplicates(subset="Metropolitan area").reset_index()

        
    population_by_region = merge["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = merge["W/L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [33]:
nba_correlation()

C:\Users\KIIT\AppData\Local\Temp\ipykernel_20920\286440073.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cities.drop([16,17,19,20,21,22,23,26,29,30,31,34,35,36,37,39,40,43,44,47,48,49,50],0,inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_20920\286440073.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NBA']=cities['NBA'].str.replace("\[.*\]","")


-0.17636350642182946

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities.drop([24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50],0,inplace=True)
    cities['MLB']=cities['MLB'].str.replace("\[.*\]","")
    cities["MLB"] = cities["MLB"].replace({"YankeesMets": "Yankees,Mets",
                                            "DodgersAngels": "Dodgers,Angels", 
                                            "CubsWhite Sox": "Cubs,White,Sox", 
                                            "GiantsAthletics": "Giants,Athletics", 
                                            "Blue Jays": "Blue,Jays",
                                            "Red Sox": "Red,Sox"})
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
    cities = cities.explode("MLB")

    mlb_df=mlb_df[mlb_df["year"]==2018]
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])

    merge=pd.merge(cities,mlb_df,left_on="MLB",right_on="team")
    merge = merge[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L","W-L%"]]
    merge["Population (2016 est.)[8]"] = merge["Population (2016 est.)[8]"].astype("float")
    merge["W-L%"]=merge["W-L%"].astype(float)

    merge.loc[merge["Metropolitan area"] == "New York City", "W-L%"] = 0.546
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.5291
    merge.loc[merge["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.525 
    merge.loc[merge["Metropolitan area"] == "Chicago", "W-L%"] = 0.482769 
    merge.loc[merge["Metropolitan area"] == "Boston", "W-L%"] = 0.666667

    merge = merge.drop_duplicates(subset="Metropolitan area").reset_index()

        
    population_by_region = merge["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = merge["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [18]:
mlb_correlation()

C:\Users\KIIT\AppData\Local\Temp\ipykernel_15004\731783672.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cities.drop([24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50],0,inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_15004\731783672.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['MLB']=cities['MLB'].str.replace("\[.*\]","")


0.15008083896523988

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities.drop([13,22,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50],0,inplace=True)
    cities['NFL']=cities['NFL'].str.replace("\[.*\]","")
    cities["NFL"] = cities["NFL"].replace({"GiantsJets": "Giants,Jets",
                                            "RamsChargers": "Rams,Chargers",
                                            "49ersRaiders": "49ers,Raiders"})
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    cities = cities.explode("NFL")

    nfl_df=nfl_df[nfl_df["year"]==2018]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.replace("*", ""))
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.replace("+", ""))
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])

    merge=pd.merge(cities,nfl_df,left_on="NFL",right_on="team")
    merge = merge[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L","W-L%"]]
    merge["Population (2016 est.)[8]"] = merge["Population (2016 est.)[8]"].astype("float")
    merge["W-L%"]=merge["W-L%"].astype(float)

    merge.loc[merge["Metropolitan area"] == "New York City", "W-L%"] = 0.2815
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.7815
    merge.loc[merge["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.25 
    merge = merge.drop_duplicates(subset="Metropolitan area").reset_index()


    
    population_by_region = merge["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = merge["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [15]:
nfl_correlation()

C:\Users\KIIT\AppData\Local\Temp\ipykernel_23092\951771597.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cities.drop([13,22,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50],0,inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_23092\951771597.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NFL']=cities['NFL'].str.replace("\[.*\]","")


0.004282141436393005

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values